In [3]:
!pip install tensorflow
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Activation
from tensorflow.keras.optimizers import RMSprop

filepath=tf.keras.utils.get_file('shakespeare','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')



1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [4]:
text=open(filepath,'rb').read().decode(encoding='utf-8').lower()

In [5]:
text

characters=sorted(set(text)) #set=>every character one time appears

char_to_index=dict((c,i)  for  i,c in enumerate(characters))
#enumerate => assigns one number to each character
#{'a':1,'f':25}
index_to_char=dict((i,c)  for  i,c in enumerate(characters))
#{1:'a'}
#Convert our text to numeric part

SEQ_LENGTH=40  #one sequence length
STEP_SIZE=3  # how many characters we gonna shift next sentence  (shift of 3 characters)

sentences=[]  # features
next_characters=[] #result/target

for i in range(0, len(text)-SEQ_LENGTH,STEP_SIZE):
  sentences.append(text[i:i+SEQ_LENGTH])
  next_characters.append(text[i+SEQ_LENGTH])


x= np.zeros((len(sentences),SEQ_LENGTH,len(characters)),dtype=np.bool)
y=np.zeros((len(sentences),len(characters)),dtype=np.bool)

for i,sentence in enumerate(sentences):
  for t,character in enumerate(sentence):
    x[i,t,char_to_index[character]]=1
  y[i,char_to_index[next_characters[i]]]=1

In [6]:
model=Sequential()
model.add(LSTM(128,input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer=RMSprop(learning_rate=0.01))

history=model.fit(x,y,batch_size=256,epochs=50)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 2.2445
Epoch 2/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.6126
Epoch 3/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 1.5073
Epoch 4/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.4606
Epoch 5/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.4249
Epoch 6/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.4000
Epoch 7/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 1.3869
Epoch 8/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.3787
Epoch 9/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 1.3578
Epoch 10/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 1.3508
Epoch 11/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.3431
Epoch 12/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.3345
Epoch 13/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 1.3314
Epoch 14/50
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.3280
Epoch 15/50
1453/1453 ━━

In [7]:

model.save('textgenerator.keras')

In [14]:
model=tf.keras.models.load_model('textgenerator.keras')

def sample(preds,temperature=1.0):   # pick the next character
  preds=np.asarray(preds).astype('float64')
  preds=np.log(preds)/temperature
  exp_preds=np.exp(preds)
  preds=exp_preds/np.sum(exp_preds)
  probas=np.random.multinomial(1,preds,1)
  return np.argmax(probas)


def generate_text(length,temperature):
  start_index=random.randint(0,len(text)-SEQ_LENGTH-1)
  generated=''
  sentence=text[start_index:start_index+SEQ_LENGTH]
  generated+=sentence
  for i in range(length):
    x=np.zeros((1,SEQ_LENGTH,len(characters)))
    for t,character in enumerate(sentence):
      x[0,t,char_to_index[character]]=1

    predictions=model.predict(x,verbose=0)[0]
    next_index=sample(predictions,temperature)
    next_character=index_to_char[next_index]

    generated+=next_character
    sentence = sentence[1:]+next_character
  return generated

print('-----------------0.2-----------')
print(generate_text(300,0.2))



print('-----------------0.4-----------')
print(generate_text(300,0.4))
print('-----------------0.6-----------')
print(generate_text(300,0.6))
print('-----------------0.8-----------')
print(generate_text(300,0.8))
print('-----------------1.0-----------')
print(generate_text(300,1.0))




-----------------0.2-----------
o england, all the world's my way.

king edward iv:
she shall i have see the peace of the strong and the comple
the complaint to the complexion to the prince
and the fair soul to the rest of the complexior
to the seal of the great proud in the street out.

sicinius:
why then the state of the street and the crown,
and there is the complexi
-----------------0.4-----------
 you have been!

queen elizabeth:
the complaint and the body and the wars,
the truth of some profition and be so heart
to the complaintmen heart the bentsworness
from the body to the wars say their sense,
and then i was her such a parise streeps of the place;
and so come to the gods'er sounded stars
for well profess to have the bishon and
-----------------0.6-----------
e provost knows our purpose and our plotted with the wars.

buckingham:
i have god so for him to the news: so so;
yet i say so affect a boar'd to thy people
in this boding paulina, and bear thee breather
that i will we sho

In [12]:
import numpy as np
import random
import tensorflow as tf

# Load model
model = tf.keras.models.load_model('textgenerator.keras')

# Your mappings (already defined previously)
# Make sure `text`, `SEQ_LENGTH`, `char_to_index`, and `index_to_char` are available

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature  # avoid log(0)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, temperature, seed_text=None):
    if seed_text:
        sentence = seed_text.lower()[-SEQ_LENGTH:]
        sentence = sentence.rjust(SEQ_LENGTH)  # pad if shorter
    else:
        start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
        sentence = text[start_index:start_index + SEQ_LENGTH]

    generated = sentence

    for i in range(length):
        x = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            if char in char_to_index:
                x[0, t, char_to_index[char]] = 1.0

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = index_to_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

    return generated

# 👇 Get user input
try:
    length = int(input("Enter number of characters to generate: "))
    temperature = float(input("Enter temperature (0.2 to 1.0): "))
    seed = input("Enter seed text (or leave blank for random): ")

    print("\n📝 Generated Text:\n")
    print(generate_text(length, temperature, seed_text=seed if seed.strip() != "" else None))
except Exception as e:
    print("❌ Error:", e)


Enter number of characters to generate: 300
Enter temperature (0.2 to 1.0): 0.2
Enter seed text (or leave blank for random): to be, or not to be: that if the question

📝 Generated Text:

o be, or not to be: that if the question
and streeps of the street and the prince with him
to the complexion of a serposion of the ground,
to be the prince of the street and the crown,
and there is the streen from the grown son,
and this the prince with him and the gross
of the street and the complexion, and the senate
hath he shall be st
